- Full Name: **Seyyed Amirmahdi Sadrzadeh**
- Student ID: **401102015**


# 🧠 Homework 4: Vision Transformer (ViT)

## 📌 Objective
In this assignment, you will **implement a Vision Transformer (ViT) from scratch** and gain a deep understanding of its core components. You will:

- 🛠️ Complete all core blocks of the Vision Transformer model.
- 🧪 Train the ViT on the CIFAR-10 image classification dataset.
- 👁️ Visualize attention maps to interpret model behavior.
- 📊 Analyze the effect of hyperparameters (e.g., patch size, depth, heads) on model performance.

---

## 📚 Learning Goals
By the end of this assignment, you should be able to:
- Explain the role of patch embeddings, self-attention, and MLP layers in ViT.
- Implement Transformer blocks without relying on high-level libraries like Hugging Face Transformers.
- Visualize self-attention maps to understand how ViT focuses on different parts of an image.
- Experiment with architectural design choices and evaluate their effects.

---

## 🧪 Dataset
We will use the **CIFAR-10** dataset, which consists of 60,000 32×32 color images in 10 classes, with 6,000 images per class. The dataset is split into 50,000 training and 10,000 test images.

---

## 🎯 Evaluation Criteria

Your final submission will be evaluated based on:
- ✅ Correct implementation of all model components.
- ✅ Accuracy of the model on CIFAR-10 test set.
- ✅ Insightfulness of attention visualizations.
- ✅ Clarity of code and documentation.
- ✅ Quality of hyperparameter analysis and discussion.





In [1]:
# ===============================
# 1.1 Setup & Imports
# ===============================
!pip install einops torch torchvision matplotlib seaborn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
import seaborn as sns
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)


Using cuda


In [3]:

# ===============================
# 1.2 Data Loading & Visualization
# ===============================
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

## To Do: Load CIFAR10:
train_ds = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_ds  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_ds, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 44.3MB/s]


In [4]:

## TO do: Visualize 8 images with their labels
classes = train_ds.classes
images, labels = next(iter(train_loader))
fig, axes = plt.subplots(1, 8, figsize=(15, 2))
for i in range(8):
    img = images[i].permute(1, 2, 0).cpu().numpy()  # Convert CHW to HWC for plt
    # Undo normalization for visualization
    img = img * [0.2470, 0.2435, 0.2616] + [0.4914, 0.4822, 0.4465]
    img = img.clip(0,1)
    axes[i].imshow(img)
    axes[i].set_title(classes[labels[i]])
    axes[i].axis('off')
plt.show()


In [5]:
# ===============================
# 1.3 Helper: pair
# ===============================
def pair(t):
    return t if isinstance(t, tuple) else (t, t)


In [6]:
# ===============================
# 1.4 FeedForward
# ===============================
## To Do: Complete the FeedForward module -> layer norm -> Linear -> GELU -> Dropout -> Linear -> Dropout
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


In [7]:

# ===============================
# 1.5 Attention
# ===============================
## To Do: Complete the Attention module
class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = heads * dim_head
        self.heads = heads
        self.scale = dim_head ** -0.5
        ## layer norm
        self.norm = nn.LayerNorm(dim)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)
        ## softmax
        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = self.norm(x)
        q, k, v = self.to_qkv(x).chunk(3, dim=-1)
        q = rearrange(q, 'b n (h d) -> b h n d', h=self.heads)
        k = rearrange(k, 'b n (h d) -> b h n d', h=self.heads)
        v = rearrange(v, 'b n (h d) -> b h n d', h=self.heads)

        ## q, k multiplication
        dots = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        ## softmax
        attn = self.attend(dots)
        ### dropout
        attn = self.dropout(attn)
        ## v multiplication
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


In [8]:

# ===============================
# 1.6 Transformer Block
# ===============================
## To Do: Complete the Transformer module
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([
            ## Attention + FeedForward
            nn.ModuleList([
                Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout),
                FeedForward(dim, mlp_dim, dropout=dropout)
            ]) for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        for attn, ff in self.layers:
            ## To do: Normal Transformer forward, nothing weird
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)


In [9]:

# ===============================
# 1.7 ViT Model
# ===============================
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes,
                 dim, depth, heads, mlp_dim,
                 pool='cls', channels=3,
                 dim_head=64, dropout=0., emb_dropout=0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        assert image_height % patch_height == 0 and image_width % patch_width == 0

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)',
                      p1=patch_height, p2=patch_width),
            nn.Linear(patch_dim, dim)
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    ## To do: complete the forward function
    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x[:, 0] if self.pool == 'cls' else x.mean(dim=1)
        return self.mlp_head(x)


In [10]:

# ===============================
# 1.8 Training Loop & Visualization
# ===============================
## To do: Complete the training loop
def train(model, epochs=10, lr=3e-4):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    model.to(device)
    losses = []
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for imgs, labels in train_loader:
            ## Forward, loss and backward here
            imgs, labels = imgs.to(device), labels.to(device)
            opt.zero_grad()
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            loss.backward()
            opt.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)

        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    plt.plot(losses)
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    # Evaluate
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")



In [12]:

# ===============================
# Run and enjoy!
# ===============================
model = ViT(
    image_size=32, patch_size=4, num_classes=10,
    dim=128, depth=6, heads=8, mlp_dim=256,
    pool='cls', channels=3, dim_head=64,
    dropout=0.1, emb_dropout=0.1
)

train(model, epochs=30)


Epoch 1/30, Loss: 1.6650
Epoch 2/30, Loss: 1.3389
Epoch 3/30, Loss: 1.2221
Epoch 4/30, Loss: 1.1418
Epoch 5/30, Loss: 1.0796
Epoch 6/30, Loss: 1.0228
Epoch 7/30, Loss: 0.9731
Epoch 8/30, Loss: 0.9259
Epoch 9/30, Loss: 0.8866
Epoch 10/30, Loss: 0.8503
Epoch 11/30, Loss: 0.8131
Epoch 12/30, Loss: 0.7820
Epoch 13/30, Loss: 0.7502
Epoch 14/30, Loss: 0.7163
Epoch 15/30, Loss: 0.6918
Epoch 16/30, Loss: 0.6576
Epoch 17/30, Loss: 0.6288
Epoch 18/30, Loss: 0.6093
Epoch 19/30, Loss: 0.5802
Epoch 20/30, Loss: 0.5540
Epoch 21/30, Loss: 0.5301
Epoch 22/30, Loss: 0.5068
Epoch 23/30, Loss: 0.4899
Epoch 24/30, Loss: 0.4734
Epoch 25/30, Loss: 0.4552
Epoch 26/30, Loss: 0.4340
Epoch 27/30, Loss: 0.4182
Epoch 28/30, Loss: 0.4060
Epoch 29/30, Loss: 0.3834
Epoch 30/30, Loss: 0.3740


Test Accuracy: 66.84%


---

## 🧠 Bonus (Optional)
- 🧩 Implement a variant of ViT using sinusoidal positional embeddings.
- 🎨 Visualize per-head attention maps across different layers.
- 🔁 Try training with different patch sizes and analyze effects.

---